In [2]:
from azureml.core import Workspace, Experiment
from azureml.pipeline.steps import PythonScriptStep

ws = Workspace.get(name="quick-starts-ws-131494")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()


Workspace name: quick-starts-ws-131494
Azure region: southcentralus
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-131494


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
vm_size = "Standard_D2_V2"
compute_name="notebook131494"
# create the cluster
compute_min_nodes=0
compute_max_nodes=4
 



### YOUR CODE HERE ###
# Checking if  compute already exist or not  if not creating a new target
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, timeout_in_minutes=20)
    # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())



In [9]:
print(compute_name)

notebook131494


In [11]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,normal,choice
import os

# Specify parameter sampler
ps = RandomParameterSampling( { 
    "--C": choice([0.00001,0.001,0.1,1,10,50,100,10000]),
     "--max_iter": choice(range(10, 200))     } )


# Specify a Policy

policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)


In [13]:
# Create a SKLearn estimator for use with train.py
from azureml.train.estimator import Estimator
from azureml.train.sklearn import SKLearn
import shutil
#shutil.copy('./train.py',source_directory)

est = SKLearn(source_directory="./", 
                compute_target=compute_target, 
                vm_size=vm_size,
                entry_script='train.py'
               )


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=8,
                                max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [15]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

exp_hyper=Experiment(ws,"hyper_test")
model=exp_hyper.submit(hyperdrive_config,show_output=True)
model.wait_for_completion(show_output=True)
best_run=model.get_best_run_by_primary_metric()
best_run.get_metrics()





RunId: HD_26c3e260-19ea-4d0e-b5a5-ff7fec7247db
Web View: https://ml.azure.com/experiments/hyper_test/runs/HD_26c3e260-19ea-4d0e-b5a5-ff7fec7247db?wsid=/subscriptions/aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee/resourcegroups/aml-quickstarts-131494/workspaces/quick-starts-ws-131494

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-23T16:31:59.620778][API][INFO]Experiment created<END>\n""<START>[2020-12-23T16:32:00.299408][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2020-12-23T16:32:00.6464891Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-12-23T16:32:00.800426][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_26c3e260-19ea-4d0e-b5a5-ff7fec7247db
Web View: https://ml.azure.com/experiments/hyper_test/runs/HD_26c3e260-19ea-4d0e-b5a5-ff7fec7247db?wsid=/subscriptions/aa7cf8e8-d23f-4

{'Regularization Strength:': 0.1, 'Max iterations:': 115, 'Accuracy': 0.8}

In [17]:
model

Experiment,Id,Type,Status,Details Page,Docs Page
hyper_test,HD_26c3e260-19ea-4d0e-b5a5-ff7fec7247db,hyperdrive,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [9]:
best_run


Experiment,Id,Type,Status,Details Page,Docs Page
hyper_test,HD_13f9eca9-b699-4896-a875-62d0567efd95_3,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [18]:
#Used RunDetails to get which run id and its parameter values
from azureml.widgets import RunDetails
RunDetails(model).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [19]:
best_run=model.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run.get_metrics()['Accuracy'])
print('\n Regularization Strength:', best_run.get_metrics()['Regularization Strength:'])
#print('\n keep probability:',parameter_values[5])
#print('\n batch size:',parameter_values[7])

Best Run Id:  HD_26c3e260-19ea-4d0e-b5a5-ff7fec7247db_1

 Accuracy: 0.8

 Regularization Strength: 0.1


In [21]:
from azureml.core.model import Model

registered_model=Model.register(workspace=ws,model_path="log_model.pkl",model_name="test")

Registering model test
